# Basic Ontology Interface Demo

this demonstrates the use of the BasicOntologyInterface which provides simplified access to local or remote
ontologies.

We demonstrate the use of different backends, but in practice you will likely only use one depending on your use case.

- pronto or sqlite for working with ontologies which you have a local copy of, and can trade startup time for generally faster operations
- ubergraph or ontobee or ols or bioportal for an ontology that has been loaded into a remote server

## Loading from obo files using Pronto

In [1]:
from oaklib.implementations.pronto.pronto_implementation import ProntoImplementation
from oaklib.resource import OntologyResource

### Local files

First we demonstrate loading from a file on the filesystem

In [2]:
resource = OntologyResource(slug='go-nucleus.obo', directory='input', local=True)
oi = ProntoImplementation(resource)

In [3]:
rels = oi.get_outgoing_relationships_by_curie('GO:0005773')
for rel, parents in rels.items():
    print(f'  {rel} ! {oi.get_label_by_curie(rel)}')
    for parent in parents:
        print(f'    {parent} ! {oi.get_label_by_curie(parent)}')

  rdfs:subClassOf ! subClassOf
    GO:0043231 ! intracellular membrane-bounded organelle
  BFO:0000050 ! part_of
    GO:0005737 ! cytoplasm


### Remote (downloading from OBO)

Next we use pronto's load from obo library feature

In [4]:
oi = ProntoImplementation(OntologyResource(local=False, slug='go.obo'))

note this slight lag in executing the command above - while this method relieves
the need to manage and synchronize files locally there is an initial network startup
penalty

In [5]:
rels = oi.get_outgoing_relationships_by_curie('GO:0005773')
for rel, parents in rels.items():
    print(f'  {rel} ! {oi.get_label_by_curie(rel)}')
    for parent in parents:
        print(f'    {parent} ! {oi.get_label_by_curie(parent)}')

  rdfs:subClassOf ! subClassOf
    GO:0043231 ! intracellular membrane-bounded organelle
  BFO:0000050 ! part of
    GO:0005737 ! cytoplasm


## SQL Database access

We can load from a SQL Database following Semantic SQL patterns (see [docs](https://github.com/cmungall/semantic-sql/#download-sqlite-dbs) for how to download ready-made sqlite dbs for all of OBO)

In [6]:
from oaklib.implementations.sqldb.sql_implementation import SqlImplementation


In [7]:
oi = SqlImplementation(OntologyResource(slug=f'sqlite:///input/go-nucleus.db'))

In [8]:
rels = oi.get_outgoing_relationships_by_curie('GO:0005773')
for rel, parents in rels.items():
    print(f'  {rel} ! {oi.get_label_by_curie(rel)}')
    for parent in parents:
        print(f'    {parent} ! {oi.get_label_by_curie(parent)}')

  BFO:0000050 ! part of
    GO:0005737 ! cytoplasm
  rdfs:subClassOf ! None
    GO:0043231 ! intracellular membrane-bounded organelle


In [9]:
for curie in oi.basic_search('intracellular'):
    print(f' MATCH: {curie} ! {oi.get_label_by_curie(curie)} ')

 MATCH: GO:0005622 ! intracellular anatomical structure 
 MATCH: GO:0043231 ! intracellular membrane-bounded organelle 
 MATCH: GO:0031090 ! organelle membrane 
 MATCH: GO:0043229 ! intracellular organelle 


### Determining which interfaces an implementation implements

In [10]:
oi.interfaces_implemented()

[oaklib.interfaces.relation_graph_interface.RelationGraphInterface,
 oaklib.interfaces.obograph_interface.OboGraphInterface,
 oaklib.interfaces.validator_interface.ValidatorInterface]

## Loading from OWL ontologies using owlfun

TODO

## Wrapping remote ontology portals

### OLS

TODO

### BioPortal

TODO

## Wrapping SPARQL Endpoints

### Ubergraph

In [11]:
from oaklib.implementations.ubergraph.ubergraph_implementation import UbergraphImplementation
oi = UbergraphImplementation()

In [12]:
rels = oi.get_outgoing_relationships_by_curie('GO:0005773')
for rel, parents in rels.items():
    print(f'  {rel} ! {oi.get_label_by_curie(rel)}')
    for parent in parents:
        print(f'    {parent} ! {oi.get_label_by_curie(parent)}')

ERROR:root:Multiple labels for BFO:0000050 = ['part_of', 'part of']


  BFO:0000050 ! part_of
    GO:0005737 ! cytoplasm
  caro#part:of ! None
    GO:0005737 ! cytoplasm
  rdfs:subClassOf ! None
    GO:0043231 ! intracellular membrane-bounded organelle


### notes

Notice some of the differences with some of the other mechanisms:

 - ubergraph includes multiple ontologies, one of which 'injects' an legacy caro#part_of relationship
 - similarly there are different injected labels for the part-of relation
 - note also that the ubergraph implementation uses the actual predicate CURIE, currently pronto uses the shortname
 
This also involves multiple iterative calls to the API which is inefficient.

In future there will be an interface for 'bigger' operations that can be implemented more efficiently

### Ontobee

Currently the ontobee implementation doesn't allow the selection of a specific ontology
within the triplestore -- instead the whole store is treated as if it were one giant
ontology with everything merged together.

This can be confusing when one ontology contains a stale part of another - e.g. if an ontology
used to have a parent term but it has since been obsoleted.

In [13]:
from oaklib.implementations.ontobee.ontobee_implementation import OntobeeImplementation
oi = OntobeeImplementation()

In [14]:
rels = oi.get_outgoing_relationships_by_curie('GO:0005773')
for rel, parents in rels.items():
    print(f'  {rel} ! {oi.get_label_by_curie(rel)}')
    for parent in parents:
        print(f'    {parent} ! {oi.get_label_by_curie(parent)}')

ERROR:root:Multiple labels for GO:0044444 = ['obsolete cytoplasmic part', '细胞质部分', 'cytoplasmic part']


  rdfs:subClassOf ! None
    GO:0044444 ! obsolete cytoplasmic part


ERROR:root:Multiple labels for GO:0043227 = ['膜结合细胞器', 'membrane-bounded organelle']
ERROR:root:Multiple labels for GO:0043231 = ['intracellular membrane-bounded organelle', '细胞内膜结合的细胞器']


    GO:0043227 ! 膜结合细胞器
    GO:0043231 ! intracellular membrane-bounded organelle


ERROR:root:Multiple labels for BFO:0000050 = ['part_of', 'is part of', 'part of', 'part_of (type level)']
ERROR:root:Multiple labels for GO:0005737 = ['cytoplasm', '细胞质']


  BFO:0000050 ! part_of
    GO:0005737 ! cytoplasm


# Graph Operations

In [16]:
oi = ProntoImplementation(OntologyResource(local=False, slug='go.obo'))

In [17]:
ancs = oi.ancestors('GO:0005773')
for anc in list(ancs):
    print(f'{anc} ! {oi.get_label_by_curie(anc)}')

GO:0005622 ! intracellular anatomical structure
GO:0110165 ! cellular anatomical entity
GO:0043229 ! intracellular organelle
GO:0043226 ! organelle
GO:0005737 ! cytoplasm
GO:0005773 ! vacuole
GO:0005575 ! cellular_component
GO:0016020 ! membrane
GO:0043227 ! membrane-bounded organelle
GO:0043231 ! intracellular membrane-bounded organelle


In [18]:
from oaklib.vocabulary.vocabulary import IS_A, PART_OF

ancs = oi.ancestors('GO:0005773', predicates=[IS_A, PART_OF])
for anc in list(ancs):
    print(f'{anc} ! {oi.get_label_by_curie(anc)}')

GO:0005622 ! intracellular anatomical structure
GO:0110165 ! cellular anatomical entity
GO:0043229 ! intracellular organelle
GO:0043226 ! organelle
GO:0005737 ! cytoplasm
GO:0005773 ! vacuole
GO:0005575 ! cellular_component
GO:0043227 ! membrane-bounded organelle
GO:0043231 ! intracellular membrane-bounded organelle


In [19]:
def render(curie):
    return f'{curie} "{oi.get_label_by_curie(curie)}"'
for s,p,o in oi.walk_up_relationship_graph('GO:0005773', predicates=[IS_A, PART_OF]):
    print(f'{render(s)} -{render(p)}-> {render(o)}')

GO:0005773 "vacuole" -rdfs:subClassOf "subClassOf"-> GO:0043231 "intracellular membrane-bounded organelle"
GO:0005773 "vacuole" -BFO:0000050 "part of"-> GO:0005737 "cytoplasm"
GO:0005737 "cytoplasm" -rdfs:subClassOf "subClassOf"-> GO:0110165 "cellular anatomical entity"
GO:0005737 "cytoplasm" -BFO:0000050 "part of"-> GO:0005622 "intracellular anatomical structure"
GO:0005622 "intracellular anatomical structure" -rdfs:subClassOf "subClassOf"-> GO:0110165 "cellular anatomical entity"
GO:0110165 "cellular anatomical entity" -rdfs:subClassOf "subClassOf"-> GO:0005575 "cellular_component"
GO:0043231 "intracellular membrane-bounded organelle" -rdfs:subClassOf "subClassOf"-> GO:0043227 "membrane-bounded organelle"
GO:0043231 "intracellular membrane-bounded organelle" -rdfs:subClassOf "subClassOf"-> GO:0043229 "intracellular organelle"
GO:0043229 "intracellular organelle" -rdfs:subClassOf "subClassOf"-> GO:0043226 "organelle"
GO:0043229 "intracellular organelle" -BFO:0000050 "part of"-> GO:000

## Rendering using GraphViz


In [20]:
from oaklib.utilities.obograph_utils import graph_to_image, default_stylemap_path
stylemap = default_stylemap_path()

In [21]:
graph = oi.ancestor_graph('GO:0005773', predicates=[IS_A, PART_OF])

In [22]:
graph_to_image(graph, ['GO:0005773'], stylemap=stylemap, imgfile='output/vacuole.png')

(node:17004) [DEP0128] DeprecationWarning: Invalid 'main' field in '/Users/cjm/repos/obographviz/node_modules/node-getopt/package.json' of './lib'. Please either fix that or report it to the module author
(Use `node --trace-deprecation ...` to show where the warning was created)


'output/vacuole.png'

### Output

![img](output/vacuole.png)